In [ ]:
import os                          
os.chdir("/content/drive/MyDrive/Colab Notebooks")
from torch import nn, cat, mul, load
%run GMF.ipynb
%run MLP.ipynb


In [ ]:
class NCF(nn.Module):
    def __init__(self, num_users, num_items, latent_dim):
        super().__init__()
        # create GMF and MLP model
        self.embedding_user_mlp = nn.Embedding(num_users, latent_dim)
        self.embedding_item_mlp = nn.Embedding(num_items, latent_dim)
        self.embedding_user_gmf = nn.Embedding(num_users, latent_dim)
        self.embedding_item_gmf = nn.Embedding(num_items, latent_dim)

        self.fc1_mlp = nn.Linear(2 * latent_dim, 32)
        self.fc2_mlp = nn.Linear(32, 16)
        self.fc3_mlp = nn.Linear(16, 8)
        self.output = nn.Linear(2 * latent_dim, 1)


        self.dropout = nn.Dropout(p=0.2)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.load_weights(num_users, num_items, latent_dim)
        
        
    def forward(self, users, movies):
        embedding_user_mlp = self.embedding_user_mlp(users)
        embedding_item_mlp = self.embedding_item_mlp(movies)
        embedding_user_gmf = self.embedding_user_gmf(users)
        embedding_item_gmf = self.embedding_item_gmf(movies)

        # GMF first 
        x_gmf = mul(embedding_user_gmf, embedding_item_gmf)

        # MLP
        x_mlp = cat([embedding_user_mlp, embedding_item_mlp], dim=1)
        x_mlp = self.dropout(self.relu(self.fc1_mlp(x_mlp)))
        x_mlp = self.dropout(self.relu(self.fc2_mlp(x_mlp)))
        x_mlp = self.fc3_mlp(x_mlp)
        # last layer doesn't need relu or dropout

        return self.sigmoid(self.output(cat([x_gmf, x_mlp], dim=1)))
        
        
    def load_weights(self, num_users, num_items, latent_dim):

        # set alpha value
        alpha = 0.5
        # initialize NCF model with weights from pretrained model 
        gmf_model = GMF(num_users, num_items, latent_dim)
        mlp_model = MLP(num_users, num_items, latent_dim)
        gmf_model.load_state_dict(load('./GMF.pt'))
        mlp_model.load_state_dict(load('./MLP2.pt'))

        self.output.weight.data = cat([alpha * gmf_model.embedding_layer.weight, (1 - alpha) * mlp_model.fc4.weight], dim=1)
        self.output.bias.data = alpha * gmf_model.embedding_layer.bias + (1-alpha)* mlp_model.fc4.bias 
        

NameError: ignored